In [1]:
import pandas as pd
import requests, re, time
from bs4 import BeautifulSoup

In [2]:
url = "https://www.amazon.com/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A2811119011%2Cn%3A7072561011&dc&ds=v1%3ALg%2BI7W8x3qOCmrDsTOUseTdkjVGEHCRYQPlZKCjiQlo&qid=1662655090&rnid=2811119011&ref=sr_nr_n_1"
mainUrl = 'https://www.amazon.com'

In [3]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.62',
            'Accept-Language': 'en-US, en;q=0.5'})

Finding children <b>recursively</b> entails getting *their* grandchildren too, as _viable_ results.

We can generalize this process for each card and collect:
- the brand name
- the model of the device
- the memory storage
- the screen/display size, and
- the connector type

Ratings by <b>*features*</b>.
> to be implemented

## Generalizaing Review Collection

In [4]:
data = pd.DataFrame(columns=['brand', 'model', 'memoryStorage','opSystem','display', 'reviews'])
pageNum = 1

%timeit
while pageNum < 11:
    currentUrl = url
    response = requests.get(url= currentUrl, headers=headers)
    
    # getting the response content

    primaryPageSoup = BeautifulSoup(markup=response.content, features='lxml')
    
    # finding the container 

    parentDiv = primaryPageSoup.find(name="div", 
                                     attrs={'class': 's-main-slot s-result-list s-search-results sg-row'})
    
    # finding the card with phone titles

    phoneDivs = parentDiv.findChildren(name="div", attrs={'data-uuid': True}, recursive=False)
    
    for phone in phoneDivs:
        
        redirectUrl = phone.find(name='a')['href']
        
        pageRedirect = requests.get(mainUrl + redirectUrl, headers=headers)
        
        redirectSoup = BeautifulSoup(markup=pageRedirect.content, features='lxml')
        
        # Grabbing link to reviews
        try:
            reviewsUrl = redirectSoup.find(name="a", 
                                           attrs={'data-hook': 'see-all-reviews-link-foot'})['href']
        except TypeError:
            # If no reviews found, no point in collecting the information
            continue
        
        try:
            # Locate table with Device Information
            infoTable = redirectSoup.find(name='table', attrs={'class': 'a-normal a-spacing-micro'})
            
            # Gather Brand Information- eg, Samsung, Iphone, One Plus
            brandRow = infoTable.find(name='tr', attrs={'class': 'a-spacing-small po-brand'})
            brand = brandRow.findChildren(name='td', recursive=False)[1].text.strip()

            # Gather Device Model Information - eg, Iphone XS, S20FE
            modelRow = infoTable.find(name='tr', attrs={'class': 'a-spacing-small po-model_name'})
            model = modelRow.findChildren(name='td', recursive=False)[1].text.strip()

            # Gather Operating System Information - eg IOS, Android 13
            opSystemrow = infoTable.find(name='tr', 
                                         attrs={'class': 'a-spacing-small po-operating_system'})
            operatingSystem = opSystemrow.findChildren(name='td', recursive=False)[1].text.strip()

            # Gather ROM information - eg, 64GB, 128GB
            memoryStorageRow = infoTable.find(name='tr', 
                                              attrs={'class': 'a-spacing-small po-memory_storage_capacity'})
            memoryStorage = memoryStorageRow.findChildren(name='td', recursive=False)[1].text.strip()

            # Gather Screen Display Size - eg, 5.8 Inches
            displaySizeRow = infoTable.find(name='tr',
                                            attrs={'class': 'a-spacing-small po-display.size'})
            displaySize = displaySizeRow.findChildren(name="td", recursive=False)[1].text.strip()
        
        except AttributeError:
            continue


        # Pagination for Reviews 
        pageCounter = 0

        while pageCounter < 5:
            # Getting request for the review page
            reviewsPage = requests.get(url=mainUrl + reviewsUrl,
                                  headers=headers)
            # Creating the Soup
            reviewsSoup = BeautifulSoup(reviewsPage.content, features='lxml')
            
            try: 
                # Obtaining the reviews parent div
                reviewsDiv = reviewsSoup.find(name="div", 
                                           attrs={'id': 'cm_cr-review_list'})

                # All reviews in their respective divs
                reviewsList = reviewsDiv.findChildren(name='div',
                                                 attrs={'data-hook': 'review'},
                                                 recursive=False)

                for div in reviewsList:
                    text = div.find(name="span", attrs={'data-hook': 'review-body'}).text.strip('\n')

                    # Append Information to DataFrame, first five column entries will be repeated per review.
                    data.loc[len(data.index)] = [brand, model, memoryStorage, operatingSystem, displaySize, text]

                # Getting next page url and updating the reviewsUrl variable
                paginationContainer = reviewsSoup.find(name="ul", attrs={'class': 'a-pagination'})
                nextReviewsUrl = paginationContainer.findChildren(name="li", recursive=False)[1].find(name="a")['href']
                reviewsUrl = nextReviewsUrl
                
            
            except AttributeError:
                break

            # Sleeping to avoid making too many requests
            time.sleep(5)

            # Updating the page counter
            pageCounter += 1
        
    # Retrieve next Page Url
    paginationSpan = primaryPageSoup.find(name='span', attrs={'class': 's-pagination-strip'})
    pageNum += 1
    print("Next Page is: {}".format(pageNum))
    nextPageUrl = paginationSpan.find(name='a',
                   attrs={'aria-label': f'Go to next page, page {str(pageNum)}'})['href']
    
    url = mainUrl + nextPageUrl

TypeError: 'NoneType' object is not subscriptable

In [5]:
data.shape

(34, 6)